In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)



c:\Users\lenovo\OneDrive - Cong ty co phan Format Vietnam JSC\Desktop\ads_enfit


In [2]:
from libs.common import *
from config.config import *

c:/Users/lenovo/OneDrive - Cong ty co phan Format Vietnam JSC/Desktop/ads_enfit


In [3]:
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}/exp3"
os.makedirs(save_dir, exist_ok=True)

In [4]:
X_train=pd.read_csv(f'{save_dir}/X_train.csv', index_col=None)
X_valid=pd.read_csv(f'{save_dir}/X_valid.csv', index_col=None)
y_train=pd.read_csv(f'{save_dir}/y_train.csv', index_col=None)
y_valid=pd.read_csv(f'{save_dir}/y_valid.csv', index_col=None)
# X_test=pd.read_csv(f'{save_dir}/X_test.csv', index_col=None)
# y_test=pd.read_csv(f'{save_dir}/y_test.csv', index_col=None)


In [5]:
# print(X_train.shape,X_valid.shape,X_test.shape, y_train.shape, y_valid.shape, y_test.shape)

In [6]:
X_valid

,row_id,county,is_business,product_type,is_consumption,lowest_price_per_mwh,highest_price_per_mwh,eic_count,installed_capacity,euros_per_mwh,...,diffuse_radiation_hl_7d,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour)
0,1472404,9,0,3,0,144.99,158.0,143.0,1646.61,449.92,...,0.0,11,7,12,2022,130,-0.352752,0.935717,-0.965926,-0.258819
1,1472405,9,0,3,1,144.99,158.0,143.0,1646.61,449.92,...,0.0,11,7,12,2022,131,-0.352752,0.935717,-0.965926,-0.258819
2,1472406,9,1,1,0,144.99,158.0,14.0,330.00,449.92,...,0.0,11,7,12,2022,132,-0.352752,0.935717,-0.965926,-0.258819
3,1472407,9,1,1,1,144.99,158.0,14.0,330.00,449.92,...,0.0,11,7,12,2022,133,-0.352752,0.935717,-0.965926,-0.258819
4,1472408,9,1,3,0,144.99,158.0,37.0,1023.08,449.92,...,0.0,11,7,12,2022,134,-0.352752,0.935717,-0.965926,-0.258819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531667,2018347,15,1,0,1,29.00,34.0,15.0,620.00,-1.29,...,0.0,31,3,5,2023,61,0.522133,-0.852864,-0.258819,0.965926
531668,2018348,15,1,1,0,29.00,34.0,20.0,624.50,-1.29,...,0.0,31,3,5,2023,62,0.522133,-0.852864,-0.258819,0.965926
531669,2018349,15,1,1,1,29.00,34.0,20.0,624.50,-1.29,...,0.0,31,3,5,2023,63,0.522133,-0.852864,-0.258819,0.965926
531670,2018350,15,1,3,0,29.00,34.0,55.0,2188.20,-1.29,...,0.0,31,3,5,2023,64,0.522133,-0.852864,-0.258819,0.965926


### Build Decision Tree and Improve in valid set

In [8]:
# Train Decision Tree model
dt_model = DecisionTreeRegressor(random_state=seed)
dt_model.fit(X_train, y_train)



In [9]:
dt_valid_predictions = dt_model.predict(X_valid)
print(dt_valid_predictions.shape)
dt_valid_mae = mean_absolute_error(y_valid, dt_valid_predictions)
dt_valid_r2 = r2_score(y_valid, dt_valid_predictions)
print(f"Decision Tree - Validation MAE: {dt_valid_mae:.3f}, Validation R2: {dt_valid_r2:.3f}")


(531672,)
Decision Tree - Validation MAE: 90.874, Validation R2: 0.918


#### Hyparameters tuning

In [10]:
param_grid_dt = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
dt_grid_search = GridSearchCV(estimator=DecisionTreeRegressor(random_state=seed),
                              param_grid=param_grid_dt,
                              scoring='neg_mean_absolute_error',
                              cv=5, n_jobs=-1, verbose=2)

dt_grid_search.fit(X_train, y_train)

print(f"Best parameters for Decision Tree: {dt_grid_search.best_params_}")

dt_best_model = dt_grid_search.best_estimator_
dump(dt_best_model, f'{save_dir}/dt_model_tuning.joblib')
dt_valid_predictions = dt_best_model.predict(X_valid)
# y_pred = scaler_y.inverse_transform(dt_valid_predictions.reshape(-1, 1))
dt_valid_mae = mean_absolute_error(y_valid, dt_valid_predictions)
dt_valid_r2 = r2_score(y_valid, dt_valid_predictions)
print(f"Decision Tree - Validation MAE: {dt_valid_mae:.3f}, Validation R2: {dt_valid_r2:.3f}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters for Decision Tree: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2}
Decision Tree - Validation MAE: 72.510, Validation R2: 0.934


In [11]:
# dt_best_model_predictions = dt_best_model.predict(X_test)
# # y_pred = scaler_y.inverse_transform(dt_best_model_predictions.reshape(-1, 1))
# dt_best_model_mae = mean_absolute_error(y_test, dt_best_model_predictions)
# dt_best_model_r2 = r2_score(y_test, dt_best_model_predictions)
# print(f"DT model - Validation MAE: {dt_best_model_mae:.3f}, Validation R2: {dt_best_model_r2:.3f}")

### Build LGBM and Improve in valid set

In [12]:
# Train LightGBM model
lgbm_model = lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.1, n_estimators=100, random_state=seed)
lgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.137107 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31964
[LightGBM] [Info] Number of data points in the train set: 1079454, number of used features: 140
[LightGBM] [Info] Start training from score 263.049682


LGBMRegressor(objective='regression', random_state=7)

In [13]:
lgbm_model_predictions = lgbm_model.predict(X_valid)
# y_pred = scaler_y.inverse_transform(lgbm_model_predictions.reshape(-1, 1))
lgbm_model_mae = mean_absolute_error(y_valid, lgbm_model_predictions)
lgbm_model_r2 = r2_score(y_valid, lgbm_model_predictions)
print(f"LGBM model - Validation MAE: {lgbm_model_mae:.3f}, Validation R2: {lgbm_model_r2:.3f}")

LGBM model - Validation MAE: 60.691, Validation R2: 0.956


#### Hyparameters tuning

In [14]:
param_grid_lgbm = {
    'num_leaves': [31, 50],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200]
}

lgb_grid_search = GridSearchCV(estimator=lgb.LGBMRegressor(objective='regression', learning_rate=0.1, n_estimators=100, random_state=seed),
                              param_grid=param_grid_lgbm,
                              scoring='neg_mean_absolute_error',
                              cv=5, n_jobs=-1, verbose=2)

lgb_grid_search.fit(X_train, y_train)


lgb_best_model = lgb_grid_search.best_estimator_
dump(lgb_best_model, f'{save_dir}/lgbm_model_tuning.joblib')
lgb_valid_predictions = lgb_best_model.predict(X_valid)
# y_pred = scaler_y.inverse_transform(lgb_valid_predictions.reshape(-1, 1))
lgb_valid_mae = mean_absolute_error(y_valid, lgb_valid_predictions)
lgb_valid_r2 = r2_score(y_valid, lgb_valid_predictions)
print(f"LGBM - Validation MAE: {lgb_valid_mae:.3f}, Validation R2: {lgb_valid_r2:.3f}")


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.112409 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31964
[LightGBM] [Info] Number of data points in the train set: 1079454, number of used features: 140
[LightGBM] [Info] Start training from score 263.049682
LGBM - Validation MAE: 58.483, Validation R2: 0.958


In [15]:
# lgb_best_model_predictions = lgb_best_model.predict(X_test)
# # y_pred = scaler_y.inverse_transform(lgb_best_model_predictions.reshape(-1, 1))
# lgb_best_model_mae = mean_absolute_error(y_test, lgb_best_model_predictions)
# lgb_best_model_r2 = r2_score(y_test, lgb_best_model_predictions)
# print(f"DT model - Validation MAE: {lgb_best_model_mae:.3f}, Validation R2: {lgb_best_model_r2:.3f}")

### Build XGBoost and Improve in valid set

In [16]:
# Train LightGBM model
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror',
    eval_metric='rmse',
    use_label_encoder=False,
    n_estimators=100,
    learning_rate=0.1,
    random_state=seed)
xgboost_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=7, ...)

In [17]:
xgboost_model_predictions = xgboost_model.predict(X_valid)
# y_pred = scaler_y.inverse_transform(xgboost_model_predictions.reshape(-1, 1))
xgboost_model_mae = mean_absolute_error(y_valid, xgboost_model_predictions)
xgboost_model_r2 = r2_score(y_valid, xgboost_model_predictions)
print(f"DT model - Validation MAE: {xgboost_model_mae:.3f}, Validation R2: {xgboost_model_r2:.3f}")

DT model - Validation MAE: 59.089, Validation R2: 0.957


### Hyparameter Tuning

In [18]:
param_grid_xgboost = {
    'min_child_weight': [3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.6, 1.0],
}

xgboost_grid_search = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror',
                                                              n_estimators=100,
                                                              learning_rate=0.01,
                                                              max_depth=3,
                                                              gamma=0.1,
                                                                eval_metric='rmse',
                                                                use_label_encoder=False,
                                                                random_state=seed),
                                                                    param_grid=param_grid_xgboost,
                                                                    scoring='neg_mean_absolute_error',
                                                                    cv=5, n_jobs=-1, verbose=2)

xgboost_grid_search.fit(X_train, y_train)


xgboost_model = xgboost_grid_search.best_estimator_
dump(xgboost_model, f'{save_dir}/xgb_model_tuning.joblib')
xgb_valid_predictions = xgboost_model.predict(X_valid)
# y_pred = scaler_y.inverse_transform(lgb_valid_predictions.reshape(-1, 1))
xgb_valid_mae = mean_absolute_error(y_valid, xgb_valid_predictions)
xgb_valid_r2 = r2_score(y_valid, xgb_valid_predictions)
print(f"LGBM - Validation MAE: {xgb_valid_mae:.3f}, Validation R2: {xgb_valid_r2:.3f}")


Fitting 5 folds for each of 8 candidates, totalling 40 fits
LGBM - Validation MAE: 173.550, Validation R2: 0.815


In [19]:
# xgboost_model_predictions = xgboost_model.predict(X_test)
# # y_pred = scaler_y.inverse_transform(xgboost_model_predictions.reshape(-1, 1))
# xgboost_model_mae = mean_absolute_error(y_test, xgboost_model_predictions)
# xgboost_model_r2 = r2_score(y_test, xgboost_model_predictions)
# print(f"DT model - Validation MAE: {xgboost_model_mae:.3f}, Validation R2: {xgboost_model_r2:.3f}")

In [20]:
print(f"Best parameters for lgbm: {xgboost_grid_search.best_params_}")
np.savez(f'{save_dir}/best_params_.npz', dt_grid_search=dt_grid_search.best_params_, lgb_grid_search = lgb_grid_search.best_params_, xgboost_grid_search = xgboost_grid_search.best_params_)


Best parameters for lgbm: {'colsample_bytree': 1.0, 'min_child_weight': 3, 'subsample': 0.8}


In [21]:
dict(np.load(f'{save_dir}/best_params_.npz',allow_pickle=True))


{'dt_grid_search': array({'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2},
       dtype=object),
 'lgb_grid_search': array({'learning_rate': 0.1, 'n_estimators': 200, 'num_leaves': 50},
       dtype=object),
 'xgboost_grid_search': array({'colsample_bytree': 1.0, 'min_child_weight': 3, 'subsample': 0.8},
       dtype=object)}